In [ ]:
import json
import numpy as np
import pandas as pd
import pyproj

In [ ]:
# load vehicle detections
car_triangulations = pd.read_csv(
    "/home/sofia_afn/Downloads/univrses_data/parked_car_triangulations_2023_04_24.csv"
)


In [ ]:
car_triangulations.head()

In [ ]:
# Get SWEREF99 (EPSG::3011) coordinates for each car detection
transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3011")

EPSG_3011_x = []
EPSG_3011_y = []
for index, row in car_triangulations.iterrows():
    y, x = transformer.transform(row["latitude"], row["longitude"])
    EPSG_3011_x.append(x)
    EPSG_3011_y.append(y)

car_triangulations["EPSG_3011_x"] = EPSG_3011_x
car_triangulations["EPSG_3011_y"] = EPSG_3011_y


In [ ]:
car_triangulations.head()

In [ ]:
# Load the parking segments
with open(
    "/home/sofia_afn//Downloads/univrses_data/permitted_parking_linestrings.json"
) as user_file:
    parking = json.load(user_file)


In [ ]:
parking

In [ ]:
# Plot all the parking segments and the car triangulations in EPSG:3011

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=[
            p[0]
            for f in parking["features"]
            for p in f["geometry"]["coordinates"] + [[None, None]]
        ],
        y=[
            p[1]
            for f in parking["features"]
            for p in f["geometry"]["coordinates"] + [[None, None]]
        ],
        name="Parking areas",
    )
)
fig.add_trace(
    go.Scatter(
        x=car_triangulations["EPSG_3011_x"],
        y=car_triangulations["EPSG_3011_y"],
        name="Car triangulations",
        mode="markers",
    )
)

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

fig.show()


In [ ]:
# choose one triangulation:

In [ ]:
triangulation = car_triangulations.iloc[[42]]

In [ ]:
triangulation_point = [
    triangulation["EPSG_3011_x"].values[0],
    triangulation["EPSG_3011_y"].values[0],
]
print(triangulation_point)


In [ ]:
# Keep parking that have a node at less than 100m
nearby_parking_ids = []
for parking_idx, p in enumerate(parking["features"]):
    # has_nearby_node = False
    for node in p["geometry"]["coordinates"]:
        dist = np.linalg.norm(np.array(triangulation_point) - np.array(node))
        if dist < 20:
            nearby_parking_ids.append(parking_idx)
            continue


In [ ]:
nearby_parking_ids

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=[
            p[0]
            for idx in nearby_parking_ids
            for p in parking["features"][idx]["geometry"]["coordinates"]
            + [[None, None]]
        ],
        y=[
            p[1]
            for idx in nearby_parking_ids
            for p in parking["features"][idx]["geometry"]["coordinates"]
            + [[None, None]]
        ],
        name="Parking areas",
    )
)
fig.add_trace(
    go.Scatter(
        x=[triangulation_point[0]],
        y=[triangulation_point[1]],
        name="Car triangulations",
        mode="markers",
    )
)

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

fig.show()


In [ ]:
# TODO calculate the between each parking area and the point


In [ ]:
triangulation

In [ ]:
triangulation_point

In [ ]:
parking["features"][1]["geometry"]["coordinates"][0][0]


In [ ]:
[
    p[0]
    for idx in nearby_parking_ids
    for p in parking["features"][idx]["geometry"]["coordinates"]
]


In [ ]:
nearby_parking_ids

In [ ]:
line = parking["features"][2677]["geometry"]["coordinates"]
line


In [ ]:
parking["features"][2677]


In [ ]:
import plotly.graph_objects as go

fig_1 = go.Figure()

fig_1.add_trace(
    go.Scatter(x=[line[0][0], line[1][0]], y=[line[0][1], line[1][1]], name="line")
)

fig_1.add_trace(
    go.Scatter(
        x=[triangulation_point[0]],
        y=[triangulation_point[1]],
        name="pointy",
        mode="markers",
    )
)

fig_1.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

fig_1.show()


In [ ]:
triangulation_point


In [ ]:
p1, p2 = np.array(line)
p2

In [ ]:
p1

In [ ]:
np.array(line)

In [ ]:
p2 - p1


In [ ]:
def project_point_on_line_segment(triangulation_point, line):
    """Calculate the projection of a point on a line segment using numpy."""
    # Define the two points of the line segment
    p1, p2 = np.array(line)

    # Calculate the vector between the two points
    v = p2 - p1

    # Normalize the vector
    v_norm = v / np.linalg.norm(v)

    # Calculate the vector between the first point of the line segment and the point we want to project
    u = triangulation_point - p1

    # Calculate the dot product of the normalized vector and the vector we just calculated
    dot_product = np.dot(u, v_norm)

    # Multiply the dot product by the normalized vector
    projection = p1 + dot_product * v_norm

    # Check if the projection is within the line segment
    if np.dot(projection - p1, v) < 0:
        projection = p1
    elif np.dot(projection - p2, v) > 0:
        projection = p2

    return projection


In [ ]:
proj = project_point_on_line_segment(triangulation_point, line)

In [ ]:
proj.tolist()

In [ ]:
proj[0]

In [ ]:
import plotly.graph_objects as go

fig_1 = go.Figure()

# line
fig_1.add_trace(
    go.Scatter(x=[line[0][0], line[1][0]], y=[line[0][1], line[1][1]], name="line")
)
# point
fig_1.add_trace(
    go.Scatter(
        x=[triangulation_point[0]],
        y=[triangulation_point[1]],
        name="pointy",
        mode="markers",
    )
)

# proj
fig_1.add_trace(
    go.Scatter(
        x=[triangulation_point[0], proj[0]],
        y=[triangulation_point[1], proj[1]],
        name="proj",
    )
)
fig_1.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

fig_1.show()
